In [10]:
import osmnx as ox
import numpy as np
import queue
import math
import priority_dict
from visualization import plot_path

In [11]:
map_graph = ox.graph.graph_from_address('Gummersbach, Steinmüllerallee 1, Germany', dist=5000, network_type='bike')

In [12]:
# Breiten- und Längengrad
origin_point = (50.985108, 7.542490)
destination_point = (51.022255, 7.562705) #Das Ziel, der Campus Gummersbach

origin = ox.distance.nearest_nodes(map_graph, origin_point[1], origin_point[0])
destination = ox.distance.nearest_nodes(map_graph, destination_point[1], destination_point[0])

In [13]:
print(origin)
print(destination)

5827857288
5969466843


In [14]:
shortest_path = ox.distance.shortest_path(map_graph, origin, destination, weight='length')

In [15]:
plot_path(map_graph, shortest_path, origin_point, destination_point)

In [16]:
def dijkstras_search(origin_key, goal_key, graph):
    """
    Finden des kürzesten Weges durch Einsatz des Dijkstra Algorithmus.

    Param:
    -----

    origin_key = Start Knoten
    goal_key = Zielpunkt
    graph = OpenStreetMap Graph

    Return:
    ------
    kürzester Pfad zum Ziel

    """
    adjaceny_list = {}
    distances = {}
    paths = {}
    for i in graph:
        adjaceny_list[i] = []
        distances[i] = float('inf')
        paths[i] = []
        for edge in graph.out_edges(i, data=True):
            out = edge[1]
            length = edge[2]['length']
            adjaceny_list[i].append([out, length]) 

    distances[origin_key] = 0

    pq = priority_dict.priority_dict()
    pq[origin_key] = 0

    while(len(pq)!=0):
        top = pq.pop_smallest()

        node = top[0]
        node_distance = top[1]

        for adj_node, adj_distance in adjaceny_list[node]:
            if(node_distance+adj_distance < distances[adj_node]):
                distances[adj_node] = node_distance + adj_distance
                pq[adj_node] = distances[adj_node]
                paths[adj_node] = paths[node] + [node]

    for i in paths:
        paths[i].append(i)
    return paths[goal_key]
    #return get_path(origin_key, goal_key, predecessors)

In [17]:
# This function follows the predecessor
# backpointers and generates the equivalent path from the
# origin as a list of vertex keys.
def get_path(origin_key, goal_key, predecessors):
    key = goal_key
    path = [goal_key]

    while key != origin_key:
        key = predecessors[key]
        path.insert(0, key)

    return path

In [18]:
path = dijkstras_search(origin, destination, map_graph)
print(path)
plot_path(map_graph, path, origin_point, destination_point)

[5827857288, 1829956474, 2589572384, 1829956477, 2589577935, 2589577930, 6959690239, 560111264, 2965563380, 33752300, 2965563412, 6777824941, 2965563347, 549386798, 3236639947, 560111275, 3705066560, 3705066581, 1704430016, 1704430019, 1704430074, 3797074112, 612920976, 26151847, 26151849, 26151853, 1928722702, 1928722710, 4740066747, 5882313585, 5882313169, 7711031487, 1704430163, 5875684250, 5901739605, 7394702748, 5882313165, 5882313164, 518772106, 518772117, 518772107, 518772067, 5091927279, 420509215, 306337851, 306338035, 306337803, 5091927228, 5091927246, 5091927245, 316243837, 365801912, 6057567282, 5969466843]
